### Tokenizer 使用

In [ ]:
!pip install transformers datasets wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 18.7 MB/s 
     |████████████████████████████████| 452 kB 74.9 MB/s 
     |████████████████████████████████| 1.9 MB 54.2 MB/s 
     |████████████████████████████████| 7.6 MB 51.1 MB/s 
     |████████████████████████████████| 182 kB 58.9 MB/s 
     |████████████████████████████████| 213 kB 65.0 MB/s 
     |████████████████████████████████| 132 kB 60.2 MB/s 
     |████████████████████████████████| 127 kB 43.7 MB/s 
     |████████████████████████████████| 184 kB 25.9 MB/s 
     |████████████████████████████████| 174 kB 52.3 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 173 kB 62.4 MB/s 
     |████████████████████████████████| 168 kB 60.9 MB/s 
     |████████████████████████████████| 168 kB 58.2 MB/s 
     |████████████████████████████████| 166 kB 58.5 MB/s 
     |███████████████████████████

In [ ]:
from transformers import AutoTokenizer

tokenizer_name = "uer/roberta-base-finetuned-jd-full-chinese"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
string_arr = [
    "店員態度極差"
]
inputs = tokenizer(string_arr, padding=True, truncation=True, return_tensors="pt")
print(inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[ 101, 2421, 1519, 2706, 2428, 3513, 2345,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


### Transformers model使用

In [ ]:
from transformers import AutoModelForSequenceClassification, RobertaForSequenceClassification

model_name = "uer/roberta-base-finetuned-jd-full-chinese"
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)
outputs = model(**inputs)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/409M [00:00<?, ?B/s]

Some weights of the model checkpoint at uer/roberta-base-finetuned-jd-full-chinese were not used when initializing RobertaForSequenceClassification: ['bert.encoder.layer.7.attention.self.key.bias', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.10.attention.self.query.bias', 'bert.encoder.layer.6.attention.self.query.bias', 'bert.encoder.layer.7.attention.self.query.bias', 'bert.encoder.layer.3.output.LayerNorm.bias', 'classifier.weight', 'bert.encoder.layer.8.output.LayerNorm.weight', 'bert.encoder.layer.6.attention.self.value.bias', 'bert.encoder.layer.3.attention.self.value.weight', 'bert.embeddings.LayerNorm.weight', 'bert.encoder.layer.10.attention.self.value.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.dense.bias', 'bert.encoder.layer.4.output.LayerNorm.weight', 'bert.encoder.layer.7.output.dense.bias', 'bert.encoder.layer.9.attention.self.key.bias', 'bert.encoder.layer.0.attention.output.dense.weight', '

In [ ]:
# print output's logits
print(outputs.logits)

tensor([[ 0.0816,  0.3115,  0.2810, -0.5602,  0.0577]],
       grad_fn=<AddmmBackward0>)


### 將結果過 softmax

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.2007, 0.2526, 0.2450, 0.1057, 0.1960]], grad_fn=<SoftmaxBackward0>)


### 查看model label

In [ ]:
model.config.id2label

{0: 'star 1', 1: 'star 2', 2: 'star 3', 3: 'star 4', 4: 'star 5'}

### 以上程式碼簡化

In [ ]:
from transformers import pipeline
model_name = "uer/roberta-base-finetuned-jd-full-chinese"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier(
    [
    "拿到肉桂捲真的是我傻眼，外帶就直接保鮮膜包起來，還是冰的，也沒有加熱什麼都沒有，也不能直接吃因為要等退冰！整顆硬梆梆的，我也是醉了，還不如去麵包店買還可以現場吃，對了，這小小硬梆梆的一顆要90元，給大家參考"
    ]
)

[{'label': 'star 1', 'score': 0.5363351106643677}]

# Tensorflow 模型加載

In [ ]:
# import tensorflow as tf
from transformers import TFAutoModel

tf_model = TFAutoModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/409M [00:00<?, ?B/s]

Some layers from the model checkpoint at uer/roberta-base-finetuned-jd-full-chinese were not used when initializing TFBertModel: ['classifier', 'dropout_37']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at uer/roberta-base-finetuned-jd-full-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### 載入dataset

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict

In [ ]:
dict0 = {
    1:0,
    2:1,
    3:2,
    4:3,
    5:4
}

In [ ]:
train_df = pd.read_csv('train.csv')
train_df = train_df.drop(columns=['Unnamed: 0'])
train_df = train_df[train_df['verse_text'].str.len() < 500]
train_df['label'] = train_df['label'].map(dict0)

In [ ]:
test_df = pd.read_csv('test.csv')
test_df = test_df.drop(columns=['Unnamed: 0'])
test_df = test_df[test_df['verse_text'].str.len() < 500]
test_df['label'] = test_df['label'].map(dict0)

In [ ]:
validation_df = pd.read_csv('validation.csv')
validation_df = validation_df.drop(columns=['Unnamed: 0'])
validation_df = validation_df[validation_df['verse_text'].str.len() < 500]
validation_df['label'] = validation_df['label'].map(dict0)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(validation_df)
# my_dataset_dict = datasets.DatasetDict({"train":train_dataset,"validation":validation_dataset ,"test":test_dataset})
my_dataset_dict = DatasetDict({"train":train_dataset,"validation":validation_dataset ,"test":test_dataset})

In [ ]:
my_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['verse_text', 'label', '__index_level_0__'],
        num_rows: 1045
    })
    validation: Dataset({
        features: ['verse_text', 'label', '__index_level_0__'],
        num_rows: 224
    })
    test: Dataset({
        features: ['verse_text', 'label', '__index_level_0__'],
        num_rows: 225
    })
})

In [ ]:
my_dataset_dict.set_format(type="pandas")
df = my_dataset_dict["train"][:]
df.head()

,verse_text,label,__index_level_0__
0,overrated，烤肉雙拼（五花松阪）豬腥味很重，端上桌就聞得到，松阪薄切缺乏口感，小菜的...,1,0
1,外帶紅棗蓮子湯$80只有一般外帶裝飯的小碗半碗，份量少到傻眼之外，味道還非常普通！！ 櫃...,1,1
2,餐點很ok，服務很差。 上主菜+高麗菜續一次後整整30分鐘沒再有服務生理。只有聽到女服務...,1,2
3,服務親切.餐點好吃環境很棒.還有現場演唱.約會的好去處,4,3
4,好心小聲告知之前他們在橋上的店，炸出的臭豆腐都沒有味道，好像拜過的食物，我們都點大辣和中辣，...,0,4


In [ ]:
labels = ["star 1","star 2","star 3","star 4","star 5"]

In [ ]:
my_dataset_dict.reset_format()

### 分詞

In [ ]:
from transformers import AutoTokenizer

model_name = "uer/roberta-base-finetuned-jd-full-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["verse_text"], padding=True, truncation=True)

In [ ]:
my_dataset_dict_encoded = my_dataset_dict.map(tokenize, batched=True, batch_size=None)
# next(iter(my_dataset_dict_encoded["train"]))

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
my_dataset_dict_encoded

DatasetDict({
    train: Dataset({
        features: ['verse_text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1045
    })
    validation: Dataset({
        features: ['verse_text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 224
    })
    test: Dataset({
        features: ['verse_text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 225
    })
})

In [ ]:
my_dataset_dict_encoded = my_dataset_dict_encoded.remove_columns('verse_text')
my_dataset_dict_encoded = my_dataset_dict_encoded.remove_columns('__index_level_0__')


### 訓練模型

In [ ]:
import torch

num_labels = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = (AutoModelForSequenceClassification
        .from_pretrained(model_name, num_labels=num_labels
        ,id2label={"0": "star 1",
                    "1": "star 2",
                    "2": "star 3",
                    "3": "star 4",
                    "4": "star 5"}
        ,label2id={"star 1": "0",
                    "star 2": "1",
                    "star 3": "2",
                    "star 4": "3",
                    "star 5": "4" })
         .to(device))

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy

batch_size = 16
logging_steps = len(my_dataset_dict_encoded["train"]) // batch_size
model_name = "Pikao_model"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=25,
                                  learning_rate=3e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  eval_steps = 50,
                                  evaluation_strategy = "steps",
                                  logging_strategy="steps",
                                  disable_tqdm=False,
                                  label_names= labels,
                                  report_to = "wandb",
                                  logging_steps=logging_steps,
                                  metric_for_best_model = 'f1',
                                  load_best_model_at_end=True)

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy

batch_size = 16
logging_steps = len(my_dataset_dict_encoded["train"]) // batch_size
model_name = "test_model"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=25,
                                  learning_rate=3e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  label_names= labels,
                                  report_to = "wandb",
                                  logging_steps=logging_steps)
                                  # load_best_model_at_end=True)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-27-75db7450f822>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_id
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=my_dataset_dict_encoded["train"],
                  eval_dataset=my_dataset_dict_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1045
  Num Epochs = 25
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1650
  Number of trainable parameters = 102271493
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,No log,No log
100,0.819700,No log
150,0.456300,No log
200,0.249600,No log
250,0.249600,No log
300,0.114200,No log
350,0.105400,No log
400,0.075600,No log
450,0.075600,No log
500,0.056200,No log


***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
Saving model checkpoint to Pikao_model/checkpoint-500
Configuration saved in Pikao_model/checkpoint-500/config.json
Model weights saved in Pikao_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in Pikao_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in Pikao_m

KeyError: ignored

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: verse_text, __index_level_0__. If verse_text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1045
  Num Epochs = 25
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1650
  Number of trainable parameters = 102271493


Step,Training Loss,Validation Loss
50,No log,No log
100,0.028300,No log
150,0.034500,No log
200,0.012400,No log
250,0.012400,No log
300,0.015900,No log
350,0.029000,No log
400,0.015200,No log
450,0.015200,No log
500,0.008600,No log


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: verse_text, __index_level_0__. If verse_text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: verse_text, __index_level_0__. If verse_text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 224
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: verse_text, __index_level_0__. If verse_text, __index_level_0__ are not expected by `BertForSequen

KeyError: ignored

In [ ]:
predictions = trainer.predict(my_dataset_dict_encoded["test"])

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: verse_text, __index_level_0__. If verse_text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 224
  Batch size = 16


Step,Training Loss,Validation Loss
50,No log,No log
100,0.082500,No log
150,0.089100,No log
200,0.078000,No log
250,0.078000,No log
300,0.053400,No log
350,0.049200,No log
400,0.058400,No log
450,0.058400,No log
500,0.017200,No log


In [ ]:
print(predictions)

PredictionOutput(predictions=(array([1.6827166, 3.1677282, 2.0795612, 2.018231 , 2.2253075, 2.0991952,
       4.060753 , 3.356965 , 3.0847116, 2.316294 , 3.648505 , 1.3874848,
       2.2663472, 1.4295458], dtype=float32), array([[-1.2677737 , -2.0109034 , -1.3985401 , -0.5647422 ,  6.9240637 ],
       [-2.2066934 , -0.3416208 ,  6.369103  , -0.55620253, -2.4654408 ],
       [-2.0277784 , -0.5535846 ,  6.292044  , -1.3592184 , -1.7699418 ],
       ...,
       [-1.8135682 , -1.2776304 , -0.6281409 ,  7.086115  , -0.3203282 ],
       [-1.3459536 , -1.8801852 , -1.2632763 , -0.86120707,  6.7824893 ],
       [-1.0158806 , -2.7013776 , -2.428445  ,  2.7062418 ,  5.8240666 ]],
      dtype=float32)), label_ids=None, metrics={'test_runtime': 7.3017, 'test_samples_per_second': 30.678, 'test_steps_per_second': 1.917})


In [ ]:
trainer.save_model('final_model_3')

Saving model checkpoint to final_model_3
Configuration saved in final_model_3/config.json
Model weights saved in final_model_3/pytorch_model.bin
tokenizer config file saved in final_model_3/tokenizer_config.json
Special tokens file saved in final_model_3/special_tokens_map.json


In [ ]:
classifier = pipeline(task= 'sentiment-analysis', 
                      model= "final_model_3")

loading configuration file final_model_3/config.json
Model config RobertaConfig {
  "_name_or_path": "final_model_3",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "star 1",
    "1": "star 2",
    "2": "star 3",
    "3": "star 4",
    "4": "star 5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "star 1": "0",
    "star 2": "1",
    "star 3": "2",
    "star 4": "3",
    "star 5": "4"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
classifier(
    [
        "一進到店內，就聞到臭味，很噁心",
        "麵的軟硬度剛好，湯也不會太鹹，不推薦大家來"
    ]
)

[{'label': 'star 1', 'score': 0.9965935349464417},
 {'label': 'star 1', 'score': 0.7985565066337585}]

In [ ]:

classifier = pipeline("sentiment-analysis", model="test_model", tokenizer=tokenizer)
classifier(
    [
    "晚上打烊過後，員工在裡面嬉鬧玩骰子，真的很想叫警察來處理",
    "煎餃很油，麵裡有大蒼蠅 = =",
    "油耗味瀰漫整個空間,清燉牛肉麵的湯頭像陽春麵+幾塊牛肉而己,不過很大碗就是！小菜沒特色~小漁干腥,泡菜硬,大鍋貼還可以！",
    "再也平凡不過的煎鍋貼，只是形狀細長，是一般四海，佳佳香的2倍長，內餡普通大眾口味，一顆也要15元，不知道為什麼知名網紅都在推薦，鍋貼6個90 / 豬肚湯60",
    "酸辣湯不錯，味道像八方。衝著大鍋貼慕名而來，很失望...若沒猜錯...大鍋貼早已煎熟放著保溫等客人。大鍋貼視覺很強烈，焦酥外皮很挑逗味蕾。若是熱的該有多好？皮不但不脆反而油膩，整口油膩感很重。慶幸內餡韭菜還行，不至於到吃不下去... 很可惜一切都溫溫的，我是願意等待現煎也不願意吃這種溫溫的鍋貼...很可惜。建議可以更換燈泡，應該可以讓用餐環境改善很多。",
    "鍋貼雖然很大，但是太油，吃起來沒有汁，料也少，還是我不喜歡的韭菜....一份六個90元，不推薦"
    ]
)

loading configuration file test_model/config.json
Model config BertConfig {
  "_name_or_path": "test_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "star 1",
    "1": "star 2",
    "2": "star 3",
    "3": "star 4",
    "4": "star 5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "star 1": "0",
    "star 2": "1",
    "star 3": "2",
    "star 4": "3",
    "star 5": "4"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

loading co

[{'label': 'star 3', 'score': 0.900388777256012},
 {'label': 'star 2', 'score': 0.998477041721344},
 {'label': 'star 3', 'score': 0.9989041090011597},
 {'label': 'star 3', 'score': 0.9990317821502686},
 {'label': 'star 2', 'score': 0.9987097978591919},
 {'label': 'star 2', 'score': 0.9991297125816345}]

In [ ]:
classifier(
    [
      "蚊蟲多環境需要加強缺少多元支付。"
    ]
)

[{'label': 'star 4', 'score': 0.9961391091346741}]

In [ ]:
a = classifier(
    [
      "蚊蟲多環境需要加強缺少多元支付。"
    ]
)

In [ ]:
test_df = pd.read_csv('test.csv')
test_model_result = classifier(test_df['verse_text'].to_list())

In [ ]:
test_d = test_df[['Unnamed: 0', 'label']]

In [ ]:
test_ans = []
for d in test_model_result:
    answer = int(d['label'][5])
    test_ans.append(answer)
test_d['ans'] = test_ans

In [ ]:
from sklearn.metrics import confusion_matrix

# Define the true labels
true_labels = test_d['label'].to_list()

# Define the predicted labels
predicted_labels = test_d['ans'].to_list()

# Calculate the confusion matrix
confusion_matrix = confusion_matrix(true_labels, predicted_labels)

# Print the confusion matrix
print(confusion_matrix)

[[ 0  7  3  0  0]
 [ 0  8  1  1  0]
 [ 0  2  5  0  0]
 [ 0  3  5 10  1]
 [ 0  0  3  1  0]]


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate the precision
precision = precision_score(true_labels, predicted_labels, average='micro')

# Calculate the recall
recall = recall_score(true_labels, predicted_labels, average='micro')

# Calculate the F1 score
f1 = f1_score(true_labels, predicted_labels, average='micro')

# Display the evaluation metrics
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Precision: 0.46
Recall: 0.46
F1 Score: 0.46


In [ ]:
test_model_result

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_df = pd.read_csv('final.csv')
mask = test_df['verse_text'].str.len() < 500
test_df = test_df[mask]
test_df

,verse_text
0,外帶泡菜鍋還不錯 但海帶芽是一打開問就是酸掉的味道 嚐起來也是東西放到酸掉的感覺 ...
1,到現場才發現店家營業時間與公開資訊不同 好好詢問，卻換來店家的冷嘲熱諷 沒吃到就算了...
2,服務態度有夠差，我點了一份牛排結果上了一份牛肉乾我還以為我畫錯了離開前還看了一下菜單沒有牛肉乾。
3,"食物普通,服務態度有待加強！ 通常進到店裡習慣洗手跟上個廁所！ 沒想到從廁所出來，竟..."
4,這價錢 但食材等級....cp值超低
...,...
995,菜色很多，又便宜，還有免費的甜湯跟湯品，在這個精華地段，有這樣CP值高的自助餐，真是附近居民...
996,目前好像只有這裡的豆漿，喝得出是自己用鐵鍋煮的那種自然焦味。 很多早餐店的豆漿是用...
997,好吃！ 肉好大塊 麵好Q彈 吃得很飽 這CP值頗高的
998,第一次吃到這種口感的蛋餅，很好吃也不用等很久 外皮薄脆香，有點像在吃蘇打餅乾的感覺 ...


In [ ]:
aaa = test_df['verse_text'].to_list()

In [ ]:
len(aaa)

6

In [ ]:
test_df

,label,1,rating,verse_text
0,2,NaN,1,"空調位置不好,較悶熱。假日人多時，上菜速度緩慢,一頓飯超過2小時。"
1,1,NaN,1,中午約莫一點在店內待了至少十分鐘以上，櫃台完全沒人，也稍微喊了一下也無人回應，請問店是有在開...
2,1,NaN,1,等了半小時說單子不見，還完全不記得我們點什麼，完全沒開始做餐，要我們重劃單，但錢都收了，竟然...
3,1,NaN,1,今天聽到一位附近工地客人跟店家要隔熱套，店家說沒有喔！ 隨客人離開後，店家居然說，手繭這麼厚...
4,2,NaN,1,22/05/13: 不好意思，這裡的徒手包含沒有戴手套喔，若是有戴手套我也不會特地來留言...
...,...,...,...,...
1495,5,NaN,5,菜色很多，又便宜，還有免費的甜湯跟湯品，在這個精華地段，有這樣CP值高的自助餐，真是附近居民...
1496,5,NaN,5,目前好像只有這裡的豆漿，喝得出是自己用鐵鍋煮的那種自然焦味。 很多早餐店的豆漿是用...
1497,5,NaN,5,好吃！ 肉好大塊 麵好Q彈 吃得很飽 這CP值頗高的
1498,5,NaN,5,第一次吃到這種口感的蛋餅，很好吃也不用等很久 外皮薄脆香，有點像在吃蘇打餅乾的感覺 ...


In [ ]:
test_model_result = classifier(aaa)

In [ ]:
test_d = test_df[['verse_text']]
test_ans = []
for d in test_model_result:
    answer = int(d['label'][5])
    test_ans.append(answer)
test_d['ans'] = test_ans

In [ ]:
test_d.to_csv('ans.csv')

In [ ]:
from sklearn.metrics import confusion_matrix

# Define the true labels
true_labels = test_d['rating'].to_list()

# Define the predicted labels
predicted_labels = test_d['ans'].to_list()

# Calculate the confusion matrix
confusion_matrix = confusion_matrix(true_labels, predicted_labels)

# Print the confusion matrix
print(confusion_matrix)

from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate the precision
precision = precision_score(true_labels, predicted_labels, average='micro')

# Calculate the recall
recall = recall_score(true_labels, predicted_labels, average='micro')

# Calculate the F1 score
f1 = f1_score(true_labels, predicted_labels, average='micro')

# Display the evaluation metrics
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

[[161  96  26   9   6]
 [ 34 123 103  34   3]
 [  8  35 106 124  27]
 [  1   5  16 162 115]
 [  0   3   7  61 229]]
Precision: 0.52
Recall: 0.52
F1 Score: 0.52
